In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
#export

from fastai import *
from fastai.vision import *
from fastai.utils.mem import *

import torch
import shutil
import random
from pathlib import Path
import json
import gc
import warnings
warnings.filterwarnings('ignore')
# fastai.torch_core.defaults.device = 'cpu'

# Photo segmetantion

## Setup

In [ ]:
# Path to annotationted mask images
path_lbl = Path("data/unet_segmetation_task/#final/train_valid_masks")
# Path to original images
path_img = Path("data/unet_segmetation_task/#final/train_valid")
# Path to test images
path_tst = Path("data/unet_segmetation_task/adepts")

# create list of RGB values in order of idx value to replace with, i.e. 0: [0,0,0], 1: [128,0,0]
rgb_list = [
    [0,0,0],
    [128,0,0]
]

## From annotated folder

In [ ]:
def unique(tensor1d):
    t, idx = np.unique(tensor1d.numpy(), return_inverse=True)
    return torch.from_numpy(t), torch.from_numpy(idx), idx   

def cut_from_folder(path_lbl, path_img, output_path):
    if not output_path.exists():
        output_path.mkdir(parents=True, exist_ok=True)

        to_separate = list(map(lambda x: path_img/f'{x.stem}.jpg', path_lbl.glob('*.png')))
        for fn in to_separate:
            shutil.copyfile(fn, output_path/f"{fn.name.split('.')[0]}.png")
        
    return output_path

In [ ]:
# Call if you need separate annotated images from not annotated images to another folder
# path_img = cut_from_folder(path_lbl, path_img, Path("data/UNET_segmentation_orig"))

In [ ]:
fnames = get_image_files(path_img)
fnames[:3]

In [ ]:
# Get from test folder all images in Path posix
tnames = get_image_files(path_tst)
tnames[:3]

In [ ]:
lbl_names = get_image_files(path_lbl)
lbl_names[:3]

In [ ]:
codes = np.loadtxt(path_lbl/'labels.txt', dtype=str,  ndmin=1); codes

In [ ]:
get_y_fn = lambda x: path_lbl/f'{x.stem}.png'

In [ ]:
img_f = fnames[0]
img = open_image(img_f)
img.show(figsize=(5, 5))
img.size

In [ ]:
mask = open_mask(get_y_fn(img_f))
mask.show(figsize=(10, 10), alpha=1)
mask.size, mask.data

# pozn.
# src_size = np.array(mask.shape[1:])
# src_size, mask.data

In [ ]:
# Get unique values 't' in mask and their indecies
t, idx_t, idx_n = unique(mask.data) 
t, idx_t

In [ ]:
size = (128, 128)
bs = 4

In [ ]:
class SegLabelListCustom(SegmentationLabelList):
    def open(self, fn): 
        return open_mask(fn, div=True)
    
class SegItemListCustom(ImageList):
    _label_cls = SegLabelListCustom

In [ ]:
# Version 1
src = (SegItemListCustom.from_folder(path_img)
       # Load in x data from folder
       .split_by_rand_pct(0.2, seed=random.randint(0, 100))
       # Split data into training and validation set 
       .label_from_func(get_y_fn, classes=codes)
#        Label data using the get_y_fn function
)

data = (src.transform(get_transforms(), size=size, tfm_y=True)
        # Flip images horizontally 
        .databunch(bs=bs)
        # Create a databunch
        .normalize(imagenet_stats)
        # Normalize for resnet
)
data.classes

In [ ]:
# data.show_batch(1, figsize=(10,10))
data.show_batch(1, figsize=(10,10), ds_type=DatasetType.Valid)

## Model

In [ ]:
# name2id = {
#     'none': 0,
#     'picture': 1
# }


name2id = {v:k for k,v in enumerate(codes, 0)}
print(name2id)

# # Void - prazdno nezanotovane miesto (Pravdepodobne)

def acc_picseg(input, target):
    target = target.squeeze(1)
    return (input.argmax(dim=1)==target).float().mean()

In [ ]:
metrics = acc_picseg
wd = 1e-2

In [ ]:
free = gpu_mem_get_free_no_cache()

print(f"using bs={bs}, have {free}MB of GPU RAM free")

In [ ]:
# create unet
learn = unet_learner(data, models.resnet34, metrics=metrics, wd=wd)

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
lr=1e-4

In [ ]:
learn.fit_one_cycle(10, slice(lr), pct_start=0.9) # train model

In [ ]:
learn.save('stage-1-128')

In [ ]:
learn.load('stage-1-128')

In [ ]:
learn.show_results(rows=3, figsize=(15, 15))

In [ ]:
img = open_image(path_img / 'SK_MRS_1272_11_r.jpg'); img

In [ ]:
prediction = learn.predict(img)[0]; prediction

In [ ]:
learn.unfreeze()

In [ ]:
lr_find(learn)
learn.recorder.plot()

In [ ]:
lr = 1e-3
lrs = slice(lr/400,lr/20); lrs

In [ ]:
learn.fit_one_cycle(20, lrs, pct_start=0.8)

In [ ]:
learn.save('stage-2-128');

In [ ]:
learn.show_results(rows=1, figsize=(15, 15))

In [ ]:
img = open_image(path_img / 'SK_MRS_1272_11_r.jpg')
prediction = learn.predict(img)[0]; prediction

In [ ]:
t, idx_t, idx_n = unique(prediction.data); t

In [ ]:
t, idx_t, idx_n  = np.unique(prediction.data.numpy(), return_inverse=True)

In [ ]:
# Validation of model 1 image

import cv2

new_img = cv2.imread('data/UNET_segmentation_orig/SK_MRS_1264_B1_r.png')
new_img = cv2.resize(new_img, (128, 128))

img = open_image('data/UNET_segmentation_orig/SK_MRS_1264_B1_r.png')
prediction = learn.predict(img)[0]

reshaped_mask = prediction.data.numpy().reshape(128, 128)

# Add 1d mask to 3d mask
mask_3d = np.dstack((reshaped_mask, reshaped_mask, reshaped_mask))
# Convert numpy array to Image
img_fastai = Image(pil2tensor(new_img * mask_3d, dtype=np.float32).div_(255)); img_fastai

In [ ]:
new_img = open_image('data/unet_segmetation_task/#2/train/SK_MRS_1309_B94_r.jpg'); new_img

In [ ]:
# Validation of model 2 image

import cv2

new_img = cv2.imread('data/unet_segmetation_task/#2/train/SK_MRS_1309_B94_r.jpg')
new_img = cv2.cvtColor(new_img, cv2.COLOR_BGR2RGB) 

new_img = cv2.resize(new_img, (128, 128))

img = open_image('data/unet_segmetation_task/#2/train/SK_MRS_1309_B94_r.jpg')
prediction = learn.predict(img)[0]

reshaped_mask = prediction.data.numpy().reshape(128, 128)

# Add 1d mask to 3d mask
mask_3d = np.dstack((reshaped_mask, reshaped_mask, reshaped_mask))
# Convert numpy array to Image
img_fastai = Image(pil2tensor(new_img * mask_3d, dtype=np.float32).div_(255)); img_fastai

# Resize Image
# img_fastai.resize((1,1024,1024))

## Bigger size

In [ ]:
#learn.destroy() # uncomment once 1.0.46 is out

size = (512, 512)

free = gpu_mem_get_free_no_cache()
# the max size of bs depends on the available GPU RAM
if free > 8200: bs=3
else:           bs=1
print(f"using bs={bs}, have {free}MB of GPU RAM free")

In [ ]:
data = (src.transform(get_transforms(), size=size, tfm_y=True)
        .databunch(bs=bs)
        .normalize(imagenet_stats))

In [ ]:
learn = unet_learner(data, models.resnet34, metrics=metrics, wd=wd)

In [ ]:
learn.load('stage-2-128');

In [ ]:
lr_find(learn)
learn.recorder.plot()

In [ ]:
lr=5e-5
learn.fit_one_cycle(10, slice(lr), pct_start=0.8)

In [ ]:
learn.save('stage-1-big-512')

In [ ]:
learn.load('stage-1-big-512')

In [ ]:
# TEST phase 1
learn.show_results(rows=1, figsize=(15, 15))

In [ ]:
img = open_image(path_img / 'SK_MRS_1266_A12_r.jpg')
prediction = learn.predict(img)[0]; prediction

In [ ]:
learn.unfreeze()

In [ ]:
# Maybe help to clear GPU memory 4me not working :(
# gc.collect()
# torch.cuda.empty_cache()
# print(gc.garbage)
gpu_mem_get_free_no_cache()
print(f"using bs={bs}, have {free}MB of GPU RAM free")

In [ ]:
lr_find(learn)
learn.recorder.plot()

In [ ]:
lr=1e-5
# lrs = slice(lr/100,lr/5); lrs

In [ ]:
# learn.fit_one_cycle(10, slice(lr), pct_start=0.8)
learn.fit_one_cycle(20, lrs)

In [ ]:
learn.show_results(rows=1, figsize=(10, 10))

In [ ]:
learn.save('stage-2-big-512')

## Cut image from original size input

## TEST model

In [ ]:
import random

In [ ]:
# learn.load('stage-4-big-1024')
learn.load('stage-2-big-512');

In [ ]:
# Get from test folder all images in Path posix
tnames = get_image_files(path_tst)
tnames[:3]

In [ ]:
idx = random.randint(0, len(tnames) - 1)
img = open_image(tnames[idx])
img.show(fig=(512, 512))

In [ ]:
prediction = learn.predict(img)[0]; prediction

In [ ]:
# Save prediction
prediction.save(f"data/unet_segmetation_task/#final/saved/{tnames[idx].stem}_m{tnames[idx].suffix}")

## Convert all adepts to segmetation model 

In [ ]:
# Get from test folder all images in Path posix
tnames = get_image_files(path_tst)
(tnames[:3], len(tnames))

In [ ]:
for name in tnames:
    img = open_image(name)
    prediction = learn.predict(img)[0]
    prediction.save(f"data/unet_segmetation_task/#final/saved/{name.stem}_m{name.suffix}")
    print(f"data/unet_segmetation_task/#final/saved/{name.stem}_m{name.suffix}")